In [1]:
# !pip install pandas
# !pip install pyarrow
# !pip install numpy
# !pip install seaborn

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, scale
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from time import time
import random
import warnings
warnings.filterwarnings("ignore")

In [3]:
# dftrain = pd.read_csv("/home/jovyan/UNSW/UNSW_NB15_training.csv") 
# dftest = pd.read_csv("/home/jovyan/UNSW/UNSW_NB15_testing.csv")
# # simplefilter(action='ignore', category=FutureWarning)

# from sklearn.preprocessing import LabelEncoder
# from sklearn.utils import shuffle


# dftrain = dftrain.drop('id', axis=1)
# dftest = dftest.drop('id', axis=1)


# dftrain = shuffle(dftrain)
# label_encoder = LabelEncoder()
# dftrain['proto'] = label_encoder.fit_transform(dftrain['proto'])
# dftrain['service'] = label_encoder.fit_transform(dftrain['service'])
# dftrain['state'] = label_encoder.fit_transform(dftrain['state'])

# dftest = shuffle(dftest)
# label_encoder = LabelEncoder()
# dftest['proto'] = label_encoder.fit_transform(dftest['proto'])
# dftest['service'] = label_encoder.fit_transform(dftest['service'])
# dftest['state'] = label_encoder.fit_transform(dftest['state'])

# #train
# #print(dftrain.loc[dftrain['target'] == 'legitimate'])
# # class_names = dftrain.target.unique()
# # dftrain=dftrain.astype('category')
# # cat_columns = dftrain.select_dtypes(['category']).columns
# # dftrain[cat_columns] = dftrain[cat_columns].apply(lambda x: x.cat.codes)
# #print(dftrain.loc[125, 'target'])
# x_columns = dftrain.columns.drop('attack_cat')
# x_train = dftrain[x_columns].values

# #test
# # class_names = dftest.target.unique()
# # dftest=dftest.astype('category')
# # cat_columns = dftest.select_dtypes(['category']).columns
# # dftest[cat_columns] = dftest[cat_columns].apply(lambda x: x.cat.codes)
# x_columns = dftest.columns.drop('attack_cat')
# x_test = dftest[x_columns].values

# y_train = dftrain['attack_cat']
# y_test = dftest['attack_cat']

# label_encoder = LabelEncoder()
# y_train = label_encoder.fit_transform(y_train)
# y_test = label_encoder.transform(y_test)


In [4]:
df = pd.read_csv("/home/jovyan/UNSW/UNSW_NB15_training.csv") 

list_drop = ['id','attack_cat']
df.drop(list_drop,axis=1,inplace=True)

df_numeric = df.select_dtypes(include=[np.number])
# df_numeric.describe(include='all')

DEBUG =0

for feature in df_numeric.columns:
    if DEBUG == 1:
        print(feature)
        print('max = '+str(df_numeric[feature].max()))
        print('75th = '+str(df_numeric[feature].quantile(0.95)))
        print('median = '+str(df_numeric[feature].median()))
        print(df_numeric[feature].max()>10*df_numeric[feature].median())
        print('----------------------------------------------------')
    if df_numeric[feature].max()>10*df_numeric[feature].median() and df_numeric[feature].max()>10 :
        df[feature] = np.where(df[feature]<df[feature].quantile(0.95), df[feature], df[feature].quantile(0.95))

df_numeric = df.select_dtypes(include=[np.number])
# df_numeric.describe(include='all')

df_numeric = df.select_dtypes(include=[np.number])
df_before = df_numeric.copy()
DEBUG = 0
for feature in df_numeric.columns:
    if DEBUG == 1:
        print(feature)
        print('nunique = '+str(df_numeric[feature].nunique()))
        print(df_numeric[feature].nunique()>50)
        print('----------------------------------------------------')
    if df_numeric[feature].nunique()>50:
        if df_numeric[feature].min()==0:
            df[feature] = np.log(df[feature]+1)
        else:
            df[feature] = np.log(df[feature])

df_numeric = df.select_dtypes(include=[np.number])

df_cat = df.select_dtypes(exclude=[np.number])
# df_cat.describe(include='all')

DEBUG = 0
for feature in df_cat.columns:
    if DEBUG == 1:
        print(feature)
        print('nunique = '+str(df_cat[feature].nunique()))
        print(df_cat[feature].nunique()>6)
        print(sum(df[feature].isin(df[feature].value_counts().head().index)))
        print('----------------------------------------------------')
    
    if df_cat[feature].nunique()>6:
        df[feature] = np.where(df[feature].isin(df[feature].value_counts().head().index), df[feature], '-')

df_cat = df.select_dtypes(exclude=[np.number])
# df_cat.describe(include='all')

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

# X.head()
feature_names = list(X.columns)
# np.shape(X)

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,2,3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

for label in list(df_cat['state'].value_counts().index)[::-1][1:]:
    feature_names.insert(0,label)
    
for label in list(df_cat['service'].value_counts().index)[::-1][1:]:
    feature_names.insert(0,label)
    
for label in list(df_cat['proto'].value_counts().index)[::-1][1:]:
    feature_names.insert(0,label)

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                     test_size = 0, 
#                                                     random_state = 0,
#                                                     stratify=y)
x_train = X
y_train = y

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train[:, 18:] = sc.fit_transform(x_train[:, 18:])
# X_test[:, 18:] = sc.transform(X_test[:, 18:])

In [5]:
df = pd.read_csv("/home/jovyan/UNSW/UNSW_NB15_testing.csv")

list_drop = ['id','attack_cat']
df.drop(list_drop,axis=1,inplace=True)

df_numeric = df.select_dtypes(include=[np.number])
# df_numeric.describe(include='all')

DEBUG =0

for feature in df_numeric.columns:
    if DEBUG == 1:
        print(feature)
        print('max = '+str(df_numeric[feature].max()))
        print('75th = '+str(df_numeric[feature].quantile(0.95)))
        print('median = '+str(df_numeric[feature].median()))
        print(df_numeric[feature].max()>10*df_numeric[feature].median())
        print('----------------------------------------------------')
    if df_numeric[feature].max()>10*df_numeric[feature].median() and df_numeric[feature].max()>10 :
        df[feature] = np.where(df[feature]<df[feature].quantile(0.95), df[feature], df[feature].quantile(0.95))

df_numeric = df.select_dtypes(include=[np.number])
# df_numeric.describe(include='all')

df_numeric = df.select_dtypes(include=[np.number])
df_before = df_numeric.copy()
DEBUG = 0
for feature in df_numeric.columns:
    if DEBUG == 1:
        print(feature)
        print('nunique = '+str(df_numeric[feature].nunique()))
        print(df_numeric[feature].nunique()>50)
        print('----------------------------------------------------')
    if df_numeric[feature].nunique()>50:
        if df_numeric[feature].min()==0:
            df[feature] = np.log(df[feature]+1)
        else:
            df[feature] = np.log(df[feature])

df_numeric = df.select_dtypes(include=[np.number])

df_cat = df.select_dtypes(exclude=[np.number])
# df_cat.describe(include='all')

DEBUG = 0
for feature in df_cat.columns:
    if DEBUG == 1:
        print(feature)
        print('nunique = '+str(df_cat[feature].nunique()))
        print(df_cat[feature].nunique()>6)
        print(sum(df[feature].isin(df[feature].value_counts().head().index)))
        print('----------------------------------------------------')
    
    if df_cat[feature].nunique()>6:
        df[feature] = np.where(df[feature].isin(df[feature].value_counts().head().index), df[feature], '-')

df_cat = df.select_dtypes(exclude=[np.number])
# df_cat.describe(include='all')

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

# X.head()
feature_names = list(X.columns)
# np.shape(X)

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,2,3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

for label in list(df_cat['state'].value_counts().index)[::-1][1:]:
    feature_names.insert(0,label)
    
for label in list(df_cat['service'].value_counts().index)[::-1][1:]:
    feature_names.insert(0,label)
    
for label in list(df_cat['proto'].value_counts().index)[::-1][1:]:
    feature_names.insert(0,label)

from sklearn.model_selection import train_test_split
x_test, x_val, y_test, y_val = train_test_split(X, y, 
                                                    test_size = 0.35, 
                                                    random_state = 0,
                                                    stratify=y)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_val[:, 18:] = sc.fit_transform(x_val[:, 18:])
x_test[:, 18:] = sc.transform(x_test[:, 18:])

In [6]:
np.save('/home/jovyan/UNSW/x_train.npy', x_train)
np.save('/home/jovyan/UNSW/y_train.npy', y_train)
np.save('/home/jovyan/UNSW/x_val.npy', x_train)
np.save('/home/jovyan/UNSW/y_val.npy', y_train)
np.save('/home/jovyan/UNSW/x_test.npy', x_test)
np.save('/home/jovyan/UNSW/y_test.npy', y_test)
print("done")

done
